# Intro to Pytorch

In [21]:
import torch
from torch.autograd import Variable
import numpy as np

# Tensors

In [8]:
# create uninitialized tensor
x = torch.Tensor(5, 3)
# create initilalized tensor
y = torch.ones(5)
z = torch.zeros(5)

# addition
x = torch.add(y, z)
torch.add(y, z, out=x)

# modify in place - any function that modifies has a trailing _
x.add_(1)


 2
 2
 2
 2
 2
[torch.FloatTensor of size 5]

In [13]:
# moving to and from numpy - they still share the same memory
a = np.ones(5)
b = torch.from_numpy(a)
b.add_(1)
print(b, a)

c = torch.ones(3)
d = c.numpy()
d += 5
print(c, d)


 2
 2
 2
 2
 2
[torch.DoubleTensor of size 5]
 [ 2.  2.  2.  2.  2.]

 6
 6
 6
[torch.FloatTensor of size 3]
 [ 6.  6.  6.]


In [20]:
# do we have a GPU?
print(torch.cuda.is_available())
x = torch.ones(5)
y = torch.zeros(5) + 7

# move things onto the GPU (this will fail if we don't have a GPU I think)
x.cuda()
y.cuda()
print(x + y)

True

 8
 8
 8
 8
 8
[torch.FloatTensor of size 5]



# Autograd

In [56]:
# Variable "Wraps a tensor and records the operations applied to it."
# http://pytorch.org/docs/0.3.0/autograd.html#torch.autograd.Variable
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

y = x + 2
print(y)

z = y * y * 3
print(z)

out = z.mean()
print(out)

# Variable.backward "Computes the gradient of current variable w.r.t. graph leaves."
# http://pytorch.org/docs/0.3.0/autograd.html#torch.autograd.Variable.backward
out.backward()
print(x.grad) # d(out)/dx. Note that you can't do this for y or z

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]

Variable containing:
 27
[torch.FloatTensor of size 1]

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]

None


In [87]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
n = 1
while y.data.norm() < 1000:
    n += 1
    y = y * 2

print(n)
print(y)
# y = 2^n * x
# dy/dx = 2^n
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)
# x.grad = 2^n scaled by the tensor we passed to y.backward. Unclear why we do this
print(x.grad)

9
Variable containing:
 378.0182
 807.5120
-659.2539
[torch.FloatTensor of size 3]

Variable containing:
  51.2000
 512.0000
   0.0512
[torch.FloatTensor of size 3]



# Neural Networks